In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from sklearn.metrics import r2_score
from scipy.stats import pearsonr, spearmanr
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.patches as mpatches
from scipy.stats import ttest_ind, ttest_1samp, ttest_rel
import matplotlib as mpl
import umap
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
import pickle
import matplotlib.ticker as mtick
import math
from itertools import combinations
import random
from sklearn.linear_model import LinearRegression
import qgrid

In [5]:
meta = pd.read_csv('../../data/E0022_P06_final_sample_map_no_control.txt',
                   sep='\t')

In [6]:
protein_ruv = pd.read_csv(
    "../../data/protein/E0022_P06_Protein_Matrix_ProNorM_no_control_update.txt",
    sep='\t').set_index('Cell_line')
name_map = pd.read_csv("../../data/misc/HUMAN_9606_idmapping.gene_prot.dat",
                       sep='\t',
                       names=['ID', 'type', 'code'])
name_map = name_map.drop_duplicates(['ID', 'type'])
name_map = pd.pivot(name_map, index='ID', columns='type', values='code').dropna()

name_map_dict = dict(zip(name_map['UniProtKB-ID'].values, name_map['Gene_Name'].values))
rna_raw = pd.read_csv("../../data/rna/rnaseq_voom.csv", index_col=0).T

rna_raw.index.name = 'SIDM'

rna_raw = rna_raw.reset_index()
rna_sample = pd.merge(rna_raw,
                      meta[['SIDM',
                            'Cell_line']].drop_duplicates()).drop(['SIDM'],
                                                                  axis=1)

rna_sample = rna_sample.set_index(['Cell_line'])

In [13]:
res_df = []
for protein in tqdm(protein_ruv.columns):
    gene = name_map_dict[protein]
    if gene not in rna_sample.columns:
        continue
    tmp_df = pd.merge(protein_ruv[[protein]], rna_sample[[gene]], on='Cell_line').dropna()
    corr, pvalue = pearsonr(tmp_df.iloc[:,0], tmp_df.iloc[:,1])
    res_df.append({'protein':protein,'gene':gene, 'pearsonr':corr,'pvalue':pvalue})
res_df = pd.DataFrame(res_df)

In [22]:
res_df.query('gene == "DKC1"')

,protein,gene,pearsonr,pvalue
739,DKC1_HUMAN,DKC1,0.252598,1.636436e-15


In [16]:
res_df.to_csv("../../result_files/rna_corr_shawn.csv", index=False)